In [ ]:
import polars   as pl
import networkx as nx
import rtsvg
rt = rtsvg.RACETrack()
from  rtsvg                   import PolarsSpringLayout
from  linknode_graph_patterns import LinkNodeGraphPatterns
import random
import copy
import time
def graphToDataFrame(_g_):
    _lu_ = {'fm':[], 'to':[]}
    for _node_ in _g_.nodes(): 
        for _nbor_ in _g_.neighbors(_node_):
            _lu_['fm'].append(_node_), _lu_['to'].append(_nbor_)
    return pl.DataFrame(_lu_), [('fm','to')]

_patterns_ = LinkNodeGraphPatterns()
_tiles_    = []
for _type_ in _patterns_.types:
    _g_             = _patterns_.createPattern(_type_)
    _df_, _relates_ = graphToDataFrame(_g_)
    _pos1_ = nx.spring_layout(_g_)
    _pos2_ = rt.springLayout(_g_, use_weights=True)
    _pos3_ = PolarsSpringLayout(_g_).results()
    _tiles_.append(rt.link(_df_, _relates_, _pos1_))
    _tiles_.append(rt.link(_df_, _relates_, _pos2_))
    _tiles_.append(rt.link(_df_, _relates_, _pos3_))
rt.table(_tiles_, per_row=3, spacer=10)

In [ ]:

from venv import create


pos = {
    'new_york': (40.7128, -74.0059),
    'los_angeles': (34.0522, -118.2437),
    'san_francisco': (37.7749, -122.4194),
    'washington_dc': (38.9072, -77.0369),
    'chicago': (41.8781, -87.6298),
    'phoenix': (33.4484, -112.0740),
    'san_diego': (32.7157, -117.1611),
    'san_antonio': (29.4241, -98.4936),
    'dallas': (32.7767, -96.7970),
    'houston': (29.7604, -95.3698),
    'austin': (30.2672, -97.7431),
    'kansas_city': (39.0997, -94.5786),
    'denver': (39.7392, -104.9903),
    'minneapolis': (44.9778, -93.2650),
    'seattle': (47.6062, -122.3321),
    'boston': (42.3601, -71.0589),
    'orlando': (28.5383, -81.3792),
    'miami': (25.7617, -80.1918),
    'atlanta': (33.7490, -84.3880),
    'tampa': (27.9506, -82.4572),
    'tallahassee': (30.4383, -84.2807),
    'salt_lake_city': (40.7608, -111.8910),
    'tempe': (33.4220, -111.9406),
    'reno': (39.5296, -119.8135),
    'las_vegas': (36.1699, -115.1398),
    'sacramento': (38.5816, -121.4944),
    'spokane': (47.6782, -117.4260),
    'boise': (43.6189, -116.2146),
    'pittsburgh': (40.4406, -79.9959),
    'philadelphia': (39.9526, -75.1652),
    'baltimore': (39.2904, -76.6122),
    'richmond': (37.54, -77.46),
    'providence': (41.8244, -71.4128),
    'bangor': (44.8378, -68.7798),
    'jacksonville': (30.3322, -81.6557),
}
def d(a, b): return 1.0 / (((pos[a][0]-pos[b][0])**2 + (pos[a][1]-pos[b][1])**2)**0.5)

_connects_ = {
    'new_york': ['boston','philadelphia', 'providence'],
    'boston': ['providence', 'new_york', 'bangor'],
    'philadelphia': ['pittsburgh', 'baltimore'],
    'pittsburgh': ['chicago', 'new_york', 'bangor'],
    'baltimore': ['washington_dc', 'pittsburgh'],
    'washington_dc': ['richmond', 'atlanta', 'chicago'],
    'chicago': ['minneapolis', 'denver', 'boise', 'kansas_city'],
    'atlanta': ['tallahassee', 'orlando', 'kansas_city', 'richmond', 'jacksonville'],
    'orlando': ['tampa', 'miami', 'tallahassee', 'jacksonville'],
    'tampa': ['miami', 'tallahassee'],
    'denver': ['salt_lake_city', 'phoenix', 'kansas_city', 'boise', 'reno', 'las_vegas'],
    'dallas': ['houston', 'san_antonio', 'denver', 'tallahassee', 'kansas_city'],
    'seattle': ['spokane', 'san_francisco'],
    'spokane': ['boise', 'minneapolis'],
    'san_francisco': ['sacramento', 'los_angeles'],
    'los_angeles': ['sacramento', 'san_diego', 'las_vegas', 'phoenix'],
    'reno': ['salt_lake_city', 'las_vegas', 'spokane', 'sacramento'],
    'phoenix':['dallas', 'san_antonio'],
    'houston': ['san_antonio'],
    'boise': ['salt_lake_city', 'reno'],
    'miami': ['jacksonville'],
}

pos_inv = {}
for k in pos.keys(): pos_inv[k] = pos[k][::-1]

_seen_ = set()
g = nx.Graph()
for _node_ in _connects_.keys():
    for _nbor_ in _connects_[_node_]:
        if _node_ == _nbor_: continue
        _seen_.add(_node_), _seen_.add(_nbor_)
        g.add_edge(_node_, _nbor_, weight=d(_node_, _nbor_))
_df_, _relates_  = graphToDataFrame(g) 
_igl_ = rt.interactiveGraphPanel(_df_, ln_params={'relationships':_relates_, 'pos':pos_inv, 'draw_labels':True}, w=800, h=600)
_igl_